<a href="https://colab.research.google.com/github/andrewfeikema/alpha-zero-general/blob/master/Othello_Train_Trial_using_AlphaZero_General.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup, install dependencies**

In [1]:
# Clone repo and install requirements

!git clone https://github.com/andrewfeikema/alpha-zero-general.git

Cloning into 'alpha-zero-general'...
remote: Enumerating objects: 1086, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 1086 (delta 7), reused 28 (delta 3), pack-reused 1051
Receiving objects: 100% (1086/1086), 424.21 MiB | 32.49 MiB/s, done.
Resolving deltas: 100% (583/583), done.
Checking out files: 100% (144/144), done.


In [2]:
%cd '/content/alpha-zero-general'

/content/alpha-zero-general


In [3]:
!git checkout -t origin/master

fatal: A branch named 'master' already exists.


In [4]:
!pip install -r docker/requirements.txt

     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 450kB 19.5MB/s 
     |████████████████████████████████| 31.2MB 140kB/s 
     |████████████████████████████████| 9.5MB 34.9MB/s 
     |████████████████████████████████| 25.3MB 86kB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=8c832963710d9f4579265a293372a54a8b2f19ced5f932299113b0868f52d1b2
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
  Created wheel for visdom: filename=visdom-0.1.7-cp37-none-any.whl size=34498 sha256=75cdb9130d638

# **Train AlphaZero**

In [48]:
import logging
import coloredlogs
from Coach import Coach
from utils import dotdict
from othello.keras.NNet import NNetWrapper
from othello.OthelloGame import OthelloGame

In [49]:
log = logging.getLogger(__name__)
coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

In [50]:
args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'checkpoint': './temp/',
    'load_model': False,
    'numItersForTrainExamplesHistory': 20,
})

In [63]:
# If you have a pre-trained model, you can load it here.
import os
if os.path.exists(os.path.join('pretrained_models', 'othello', 'keras', '8x8', 'checkpoint.index')):
  print ("Using best pre-existing model")
  args['load_model'] = True
  args['load_folder_file'] = ('pretrained_models/othello/keras/8x8','checkpoint.index')
else:
  print ("Not using best pre-existing model")

Using best pre-existing model


In [59]:
# Set very low iterations to let this notebook run in its entirety.

# In reality, training a model, even as simple as the one for Dots and Boxes, can take several hours or days.
args['numIters'] = 1
args['numEps'] = 1
args['arenaCompare'] = 2

In [60]:
game = OthelloGame(n=8)

In [61]:
nnet = NNetWrapper(game)

In [64]:
if args.load_model:
    print('Loading checkpoint "{}/{}"...'.format(args.load_folder_file[0], args.load_folder_file[1]))
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
else:
    print('Not loading a checkpoint.')

Loading checkpoint "pretrained_models/othello/keras/8x8/checkpoint.index"...


AssertionError: ignored

In [28]:
coach = Coach(game, nnet, args)

In [29]:
%time coach.learn()

2021-05-12 02:33:22 ef2865b44f7b Coach[72] INFO Starting Iter #1 ...

Self Play: 100%|██████████| 1/1 [00:53<00:00, 53.60s/it]


Checkpoint Directory exists! 
./temp/ temp.pth.tar


TypeError: ignored

In [65]:
# Checkpoints and best model (if found) will be saved in this folder
%ls /content/alpha-zero-general/temp/

checkpoint                     temp.pth.tar.data-00000-of-00001
checkpoint_0.pth.tar.examples  temp.pth.tar.index


In [ ]:
!git pull origin

In [56]:
!cp /content/alpha-zero-general/temp/temp.pth.tar.index pretrained_models/othello/keras/8x8/checkpoint.index

In [37]:
!touch pretrained_models/othello/keras/8x8/best.pth.tar